In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import requests
from bs4 import BeautifulSoup
import nltk

In [26]:
url = "https://www.antaranews.com/" #menentukan url

In [27]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser') #Isi teks dari respons HTTP yang diterima dari server web setelah melakukan permintaan GET.
# pages = soup.findAll('article', {'class' :'list-content__item'})
first_page = soup.findAll('li',"dropdown mega-full menu-color1") #menemukan semua list yang berisi link kategori

# print(first_page)
save_categori = []
for links in first_page:
  categori = links.find('a').get('href')
  save_categori.append(categori)
save_categori

['https://www.antaranews.com/',
 'https://www.antaranews.com/politik',
 'https://www.antaranews.com/hukum',
 'https://www.antaranews.com/ekonomi',
 'https://www.antaranews.com/metro',
 'https://www.antaranews.com/sepakbola',
 'https://www.antaranews.com/olahraga',
 'https://www.antaranews.com/humaniora',
 'https://www.antaranews.com/lifestyle',
 'https://www.antaranews.com/hiburan',
 'https://www.antaranews.com/dunia',
 'https://en.antaranews.com']

In [29]:
categori_search = [save_categori[2],save_categori[4],save_categori[7]] #categori yang akan disearch terdapat pada indeks 2,4,7
categori_search

['https://www.antaranews.com/hukum',
 'https://www.antaranews.com/metro',
 'https://www.antaranews.com/humaniora']

In [30]:
datas=[]

for ipages in range(1,25):
  for beritas in categori_search:
    # print(beritas)
    response_berita = requests.get(beritas+"/"+str(ipages))
    namecategori = beritas.split("/")
    soup_berita = BeautifulSoup(response_berita.text, 'html.parser')
    pages_berita = soup_berita.findAll('article', {'class' :'simple-post simple-big clearfix'})

    i = 0
    for items in pages_berita:
      i += 1
      get_link_in = items.find("a").get("href")
      response_artikel = requests.get(get_link_in)
      soup_artikel = BeautifulSoup(response_artikel.text, 'html.parser')

      cek_judul = soup_artikel.findAll("h1","post-title")
      if cek_judul:
        judul = soup_artikel.find("h1","post-title").text
      else:
        judul=""

      label = namecategori[-1]

      try:
        date = soup_artikel.find("span", "article-date").text
      except AttributeError:
          date = "Data tanggal tidak ditemukan"
      cek_baca_juga = soup_artikel.findAll("span","baca-juga")
      trash1 = ""
      if cek_baca_juga:
        for bacas in cek_baca_juga:
          text_trash = bacas.text
          trash1 += text_trash + ' '
      else:
        trash1 = ""

      artikels = soup_artikel.find_all('div', {'class' :'post-content clearfix'})
      if artikels:
        artikel_content = soup_artikel.find('div', {'class' :'post-content clearfix'}).text
        artikel = artikel_content.replace("\n"," ").replace("\t"," ").replace("\r"," ").replace(trash1,"").replace("\xa0","")
      else:
        artikel = ""
      cek_author = soup_artikel.findAll("p","text-muted small mt10")
      if cek_author:
        author = soup_artikel.find("p","text-muted small mt10").text.replace("\t\t"," ")
      else:
        author = ""
      datas.append({
      'Tanggal' : date,
      'Penulis':author,
      'Judul': judul,
      'Artikel': artikel,
      'Label' :  label
          })


In [31]:
len(datas)

1080

In [34]:
# Menyimpan data dalam bentuk CSV
import csv
csv_filename = '/content/drive/MyDrive/Colab Notebooks/dataset_antaranews.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Tanggal', 'Penulis', 'Judul', 'Artikel', 'Label']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    for entry in datas:
        csv_writer.writerow(entry)

print(f"Data telah disimpan dalam file {csv_filename}")

Data telah disimpan dalam file /content/drive/MyDrive/Colab Notebooks/dataset_antaranews.csv
